## Problem Statement


Rocket league is regarded as a video game with very slow improvement rates. Whilst easy to pick up and enjoy, the highly mechanical and precise nature of the game leads to very slow progression at high ranks. Professional e-sports players are constantly pushing the skill ceiling in terms of mechanical talent and tactical mastery, and can be used as examples for improvement and furthermore as play-style case studies for developing players, particularly at mid to high ranks.
This project uses 3 pro players, selected as top-level examples of 3 distinct play-styles:
- MonkeyMoon: 
    - Highly efficient with boost, never out of the game because of this
    - Very strong decision making - only challenging in favourable situations in order to save boost
- Vatira: 
    - ‘3rd man’ position, largely playing with a high amount of boost, able to use his ability to capitalise on space created by teammates
    - Plays close to teammates and pressure the opposition collectively
- Oski: 
    - ‘Ballchaser’, uses a large amount of boost to play as fast as possible, aiming to beat opposition to the ball or take out multiple players in one play
    - Often leaves teammates exposed, however looks to create more opportunities than conceded over a game
---
An ExtraTrees classification model has been trained on replay data from professional tournaments for each of these players, and can be used to predict which play-style a player most closely fits from a submitted 3v3 replay ID and player ID.
- Visuals highlighting key differences to each play-style can also be viewed, as well as the probabilities of each play-style being selected.
---
Furthermore, a regression model has been produced to estimate the rank of a player given the stats from a 3v3 replay ID and player ID

In [2]:
import requests
import json
import pandas as pd

import time

In [14]:
res = requests.get('https://ballchasing.com/api/replays/',
                  headers={
                      'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'},
                  params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'bronze-1'
                  })

In [15]:
data_1 = res.json()

In [16]:
data_1['list'][-1]['date']

'2023-04-30T12:44:01+02:00'

In [17]:
len(data_1['list'])

50

In [18]:
for i in range(len(data_1['list'])):
    print(data_1['list'][i]['id'])
    print(data_1['list'][i]['date'])

19e70650-5ecd-4268-b09c-bbce5d234164
2023-04-30T22:52:30+12:00
2118bbd8-5bda-4570-ac86-3809a3130f9e
2023-04-30T04:52:28-06:00
f81fc6b7-6ac9-4c84-a7a0-ef4a4652e060
2023-04-30T20:54:44+10:00
b961e7c2-47e9-4396-98e3-13fc29aa66ad
2023-04-30T03:52:27-07:00
fb2af289-4a30-471c-a636-cc6f5b9e5019
2023-04-30T20:52:07+10:00
f4a55884-4560-47c7-9feb-2d47937dd5e9
2023-04-30T20:22:07+09:30
9e257a0c-b7c2-43bd-ab49-3480dd3d6151
2023-04-30T20:52:07+10:00
a032e9d5-e713-422a-81e2-2b6b1f7328c3
2023-04-30T20:51:50+10:00
432e21b3-e5ad-4204-abc0-132d8b811f3e
2023-04-30T13:51:43+03:00
5034f2bc-01ed-40e1-ae80-d84c6e0eb7e2
2023-04-30T14:51:42+04:00
86c36199-578f-4e01-886f-4f2c58e4e159
2023-04-30T19:51:09+09:00
63dc72ac-a762-489f-b996-0f562c2c930c
2023-04-30T12:50:56+02:00
c34b6445-dcee-4627-946a-610b9a0b24d7
2023-04-30T12:50:46+02:00
2caeb53a-b31e-4c71-a23d-73b4c09db695
2023-04-30T12:50:35+02:00
2d87ee01-18ea-406b-a833-6cc356df05cf
2023-04-30T11:50:20+01:00
d87a72ef-392e-4419-bc2f-02369b430be0
2023-04-30T11:50:1

In [8]:
res = requests.get('https://ballchasing.com/api/replays/3c601398-e906-43ae-967f-bfea448b908f',
                  headers={
                      'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

data = res.json()
data

{'id': '3c601398-e906-43ae-967f-bfea448b908f',
 'link': 'https://ballchasing.com/api/replays/3c601398-e906-43ae-967f-bfea448b908f',
 'created': '2023-04-29T21:28:50.818007Z',
 'uploader': {'steam_id': '76561198156730186',
  'name': 'hidnvalley',
  'profile_url': 'https://steamcommunity.com/id/hidnvalley/',
  'avatar': 'https://avatars.akamai.steamstatic.com/52247eef5bf64f2f8a3fc1798cd85ac5b020ba11.jpg'},
 'status': 'ok',
 'rocket_league_id': 'FC48919D4E59843D65D90D971411CFE3',
 'match_guid': '241BDF3A11ECA1BCE1EAC3812DC1A71B',
 'title': 'mad combak',
 'recorder': 'AyoDos',
 'map_code': 'stadium_day_p',
 'match_type': 'Online',
 'team_size': 3,
 'playlist_id': 'ranked-standard',
 'duration': 393,
 'overtime': True,
 'overtime_seconds': 23,
 'season': 6,
 'season_type': 'free2play',
 'date': '2022-03-11T22:31:17-06:00',
 'date_has_timezone': True,
 'visibility': 'public',
 'min_rank': {'id': 'diamond-3',
  'tier': 15,
  'division': 2,
  'name': 'Diamond III Division 2'},
 'max_rank': {'i

In [21]:
params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'bronze-1'
                  }

In [226]:
def get_replays(num_replays, params, auth_key, collected_replays = 0, id_list = []):
    time.sleep(0.25)
    res = requests.get('https://ballchasing.com/api/replays/',
                  headers={
                      'Authorization': auth_key},
                  params = params
                  )
    data = res.json()
    collected_replays += len(data['list'])
    print(collected_replays, num_replays)
    if collected_replays <= num_replays:
        for i in range(len(data['list'])):
            id_list.append(data['list'][i]['id'])
            replay_end_date = data['list'][-1]['date']
            params['replay-date-before'] = replay_end_date
            
                
        get_replays(num_replays = num_replays, 
                    params = params, 
                    auth_key = auth_key,
                    collected_replays = collected_replays,
                    id_list = id_list
                   )
                
    return id_list



def get_all_players(id_list, auth_key):
    
    errors = 0
    
    res = requests.get(f'https://ballchasing.com/api/replays/{id_list[0]}',
                  headers={
                      'Authorization': auth_key}
                          )
    data = res.json()
    
    player_data = data['orange']['players'][0]['stats']
    player_data = dict(player_data, **player_data['core'])
    player_data = dict(player_data, **player_data['boost'])
    player_data = dict(player_data, **player_data['movement'])
    player_data = dict(player_data, **player_data['positioning'])
    player_data = dict(player_data, **player_data['demo'])
    del player_data['core']
    del player_data['boost']
    del player_data['movement']
    del player_data['positioning']
    del player_data['demo']

#     player_data['title'] = data['title']
    player_data['player_rank'] = ''
    
    df = pd.DataFrame(columns = player_data.keys())
    
    for i, s in enumerate(id_list):
        if i % 100 == 0:
            print(f"{i} id's have been checked, {df.shape[0]} replays collected")
        time.sleep(0.8)
        res = requests.get(f'https://ballchasing.com/api/replays/{s}',
                  headers={
                      'Authorization': auth_key}
                          )
        data = res.json()    
#         print(data['blue'])
        if 'orange' not in data.keys():
            print(f"match {i} failed to retrieve data, {df.shape[0]} replays collected")
            errors += 1
            continue
        try:
            
            if len(data['orange']['players']) != 3 or len(data['blue']['players']) != 3:
                print(f"match {i} not 3V3")
                errors += 1
                continue
        except KeyError:
                errors += 1
                print(f"KeyError at match {i}, errors = {errors}")
                continue
        
        if df.shape[0] % 5 == 0:
            print(f"{df.shape[0]} games collected, {errors} errors")
                
#         try:
        for j in range(len(data['orange']['players'])):
            try:
                
                player_data = data['orange']['players'][j]['stats']

                player_data = dict(player_data, **player_data['core'])
                player_data = dict(player_data, **player_data['boost'])
                player_data = dict(player_data, **player_data['movement'])
                player_data = dict(player_data, **player_data['positioning'])
                player_data = dict(player_data, **player_data['demo'])
                del player_data['core']
                del player_data['boost']
                del player_data['movement']
                del player_data['positioning']
                del player_data['demo']

                player_data['id'] = data['orange']['players'][j]['id']['id']
                player_data['player_rank'] = data['blue']['players'][j]['rank']['name']
                df.loc[player_data['id'], :] = player_data
            except KeyError:
                print(f"KeyError in id: {i} for orange player {j}")
                errors += 1

        for k in range(len(data['blue']['players'])):
            try:
                      
                player_data = data['blue']['players'][k]['stats']

                player_data = dict(player_data, **player_data['core'])
                player_data = dict(player_data, **player_data['boost'])
                player_data = dict(player_data, **player_data['movement'])
                player_data = dict(player_data, **player_data['positioning'])
                player_data = dict(player_data, **player_data['demo'])
                del player_data['core']
                del player_data['boost']
                del player_data['movement']
                del player_data['positioning']
                del player_data['demo']

                player_data['id'] = data['orange']['players'][k]['id']['id']
                player_data['player_rank'] = data['blue']['players'][j]['rank']['name']
                df.loc[player_data['id'], :] = player_data

            except KeyError:
                print(f"KeyError in id: {i} for blue player {k}")
                errors += 1
                
#         except KeyError:
#             errors += 1
#             print(f'KeyError in match {i}, errors = {errors}')
            

    return df, print(f"finished! {df.shape[0]} replays collected, errors = {errors}")

In [210]:
bronze_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'bronze-1',
                      'max-rank' : 'bronze-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [200]:
silver_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'silver-1',
                      'max-rank' : 'silver-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [203]:
gold_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'gold-1',
                      'max-rank' : 'gold-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [204]:
platinum_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'platinum-1',
                      'max-rank' : 'platinum-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

diamond_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'diamond-1',
                      'max-rank' : 'diamond-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

champion_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'champion-1',
                      'max-rank' : 'champion-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

gc_ssl_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'grand-champion'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [211]:
len(set(bronze_ids)), len(set(silver_ids)), len(set(gold_ids)), len(set(platinum_ids)), len(set(diamond_ids)), len(set(champion_ids)), len(set(gc_ssl_ids)),   

(422, 371, 468, 491, 413, 477, 482)

In [212]:
ids = bronze_ids + silver_ids + gold_ids + platinum_ids + diamond_ids + champion_ids + gc_ssl_ids

In [216]:
ids = list(set(ids))

In [229]:
res = requests.get(f'https://ballchasing.com/api/replays/{ids[1536]}',
                  headers={
                      'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

data = res.json()
data['blue']['players'][2]

IndexError: list index out of range

In [90]:
len(replay_id_list)

10000

In [230]:
replays = get_all_players(id_list = ids, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

0 id's have been checked, 0 replays collected
0 games collected, 0 errors
KeyError in id: 2 for orange player 0
KeyError in id: 2 for orange player 1
15 games collected, 2 errors
KeyError in id: 5 for orange player 2
KeyError in id: 5 for blue player 0
KeyError in id: 5 for blue player 1
KeyError in id: 5 for blue player 2
20 games collected, 6 errors
KeyError in id: 10 for orange player 1
KeyError in id: 10 for orange player 2
KeyError in id: 10 for blue player 0
KeyError in id: 10 for blue player 1
KeyError in id: 10 for blue player 2
30 games collected, 11 errors
45 games collected, 11 errors
KeyError in id: 17 for orange player 0
KeyError in id: 17 for orange player 2
KeyError in id: 17 for blue player 0
KeyError in id: 17 for blue player 1
KeyError in id: 17 for blue player 2
KeyError in id: 18 for orange player 0
KeyError in id: 18 for orange player 1
KeyError in id: 19 for orange player 0
KeyError in id: 19 for orange player 1
KeyError in id: 19 for orange player 2
KeyError in i

215 games collected, 200 errors
KeyError in id: 99 for orange player 1
100 id's have been checked, 218 replays collected
KeyError in id: 102 for orange player 0
KeyError in id: 103 for orange player 1
230 games collected, 203 errors
KeyError in id: 105 for orange player 0
KeyError in id: 105 for orange player 2
KeyError in id: 105 for blue player 0
KeyError in id: 105 for blue player 1
KeyError in id: 105 for blue player 2
KeyError in id: 106 for orange player 0
KeyError in id: 106 for orange player 1
KeyError in id: 106 for orange player 2
KeyError in id: 106 for blue player 0
KeyError in id: 106 for blue player 1
KeyError in id: 106 for blue player 2
KeyError in id: 108 for orange player 0
KeyError in id: 109 for orange player 0
KeyError in id: 109 for orange player 2
KeyError in id: 109 for blue player 0
KeyError in id: 109 for blue player 1
KeyError in id: 109 for blue player 2
240 games collected, 220 errors
KeyError in id: 110 for orange player 0
KeyError in id: 110 for orange pl

455 games collected, 395 errors
KeyError in id: 206 for orange player 0
KeyError in id: 206 for orange player 1
KeyError in id: 206 for orange player 2
KeyError in id: 206 for blue player 0
KeyError in id: 206 for blue player 1
KeyError in id: 206 for blue player 2
460 games collected, 401 errors
KeyError in id: 208 for orange player 0
KeyError in id: 208 for orange player 1
KeyError in id: 209 for orange player 0
KeyError in id: 209 for orange player 1
KeyError in id: 209 for orange player 2
KeyError in id: 209 for blue player 0
KeyError in id: 209 for blue player 1
KeyError in id: 209 for blue player 2
KeyError in id: 213 for orange player 0
KeyError in id: 213 for orange player 1
KeyError in id: 213 for orange player 2
KeyError in id: 213 for blue player 0
KeyError in id: 213 for blue player 1
KeyError in id: 213 for blue player 2
KeyError in id: 215 for orange player 2
KeyError in id: 215 for blue player 0
KeyError in id: 215 for blue player 1
KeyError in id: 215 for blue player 2


KeyError in id: 315 for orange player 0
KeyError in id: 315 for orange player 2
KeyError in id: 315 for blue player 0
KeyError in id: 315 for blue player 1
KeyError in id: 315 for blue player 2
KeyError in id: 316 for orange player 1
KeyError in id: 316 for orange player 2
KeyError in id: 316 for blue player 0
KeyError in id: 316 for blue player 1
KeyError in id: 316 for blue player 2
685 games collected, 601 errors
KeyError in id: 317 for orange player 0
KeyError in id: 318 for orange player 0
KeyError in id: 318 for orange player 1
KeyError in id: 320 for orange player 1
KeyError in id: 320 for orange player 2
KeyError in id: 320 for blue player 0
KeyError in id: 320 for blue player 1
KeyError in id: 320 for blue player 2
695 games collected, 609 errors
700 games collected, 609 errors
KeyError in id: 323 for orange player 0
KeyError in id: 323 for orange player 1
KeyError in id: 323 for orange player 2
KeyError in id: 323 for blue player 0
KeyError in id: 323 for blue player 1
KeyErr

870 games collected, 787 errors
KeyError in id: 410 for orange player 1
KeyError in id: 411 for orange player 0
KeyError in id: 411 for orange player 1
KeyError in id: 414 for orange player 1
KeyError in id: 414 for orange player 2
KeyError in id: 414 for blue player 0
KeyError in id: 414 for blue player 1
KeyError in id: 414 for blue player 2
KeyError in id: 419 for orange player 0
KeyError in id: 419 for orange player 1
KeyError in id: 419 for orange player 2
KeyError in id: 419 for blue player 0
KeyError in id: 419 for blue player 1
KeyError in id: 419 for blue player 2
KeyError in id: 420 for orange player 0
KeyError in id: 420 for orange player 1
KeyError in id: 421 for orange player 0
KeyError in id: 421 for orange player 2
KeyError in id: 421 for blue player 0
KeyError in id: 421 for blue player 1
KeyError in id: 421 for blue player 2
KeyError in id: 422 for orange player 1
895 games collected, 809 errors
KeyError in id: 423 for orange player 0
KeyError in id: 424 for orange pla

...

KeyError in id: 2736 for orange player 0
4580 games collected, 5960 errors
KeyError in id: 2737 for orange player 0
KeyError in id: 2737 for orange player 1
KeyError in id: 2737 for orange player 2
KeyError in id: 2737 for blue player 0
KeyError in id: 2737 for blue player 1
KeyError in id: 2737 for blue player 2
4580 games collected, 5966 errors
4585 games collected, 5966 errors
KeyError in id: 2741 for orange player 1
4590 games collected, 5967 errors
4590 games collected, 5967 errors
KeyError in id: 2745 for orange player 0
KeyError in id: 2746 for orange player 0
KeyError in id: 2746 for orange player 2
KeyError in id: 2746 for blue player 0
KeyError in id: 2746 for blue player 1
KeyError in id: 2746 for blue player 2
KeyError in id: 2747 for orange player 0
KeyError in id: 2747 for orange player 1
KeyError in id: 2747 for orange player 2
KeyError in id: 2747 for blue player 0
KeyError in id: 2747 for blue player 1
KeyError in id: 2747 for blue player 2
KeyError in id: 2749 for ora

4720 games collected, 6146 errors
match 2837 not 3V3
KeyError in id: 2838 for orange player 0
KeyError in id: 2838 for orange player 1
KeyError in id: 2838 for orange player 2
KeyError in id: 2838 for blue player 0
KeyError in id: 2838 for blue player 1
KeyError in id: 2838 for blue player 2
KeyError in id: 2840 for orange player 0
KeyError in id: 2840 for orange player 1
KeyError in id: 2842 for orange player 1
KeyError in id: 2844 for orange player 0
KeyError in id: 2844 for orange player 2
KeyError in id: 2844 for blue player 0
KeyError in id: 2844 for blue player 1
KeyError in id: 2844 for blue player 2
KeyError in id: 2845 for orange player 0
KeyError in id: 2846 for orange player 0
KeyError in id: 2847 for orange player 0
KeyError in id: 2847 for orange player 1
KeyError in id: 2848 for orange player 0
KeyError in id: 2848 for orange player 1
KeyError in id: 2849 for orange player 0
KeyError in id: 2849 for orange player 1
KeyError in id: 2849 for orange player 2
KeyError in id: 

4825 games collected, 6336 errors
KeyError in id: 2910 for orange player 0
KeyError in id: 2912 for orange player 0
KeyError in id: 2912 for orange player 1
KeyError in id: 2912 for orange player 2
KeyError in id: 2912 for blue player 0
KeyError in id: 2912 for blue player 1
KeyError in id: 2912 for blue player 2
4830 games collected, 6343 errors
4835 games collected, 6343 errors
KeyError in id: 2917 for orange player 0
KeyError in id: 2917 for orange player 1
KeyError in id: 2918 for orange player 1
KeyError in id: 2918 for orange player 2
KeyError in id: 2918 for blue player 0
KeyError in id: 2918 for blue player 1
KeyError in id: 2918 for blue player 2
KeyError in id: 2919 for orange player 1
KeyError in id: 2919 for orange player 2
KeyError in id: 2919 for blue player 0
KeyError in id: 2919 for blue player 1
KeyError in id: 2919 for blue player 2
4840 games collected, 6355 errors
KeyError in id: 2921 for orange player 0
KeyError in id: 2921 for orange player 1
match 2924 not 3V3
Ke

4975 games collected, 6525 errors
KeyError in id: 3016 for orange player 0
4980 games collected, 6526 errors
4980 games collected, 6526 errors
KeyError in id: 3019 for orange player 0
KeyError in id: 3019 for orange player 2
KeyError in id: 3019 for blue player 0
KeyError in id: 3019 for blue player 1
KeyError in id: 3019 for blue player 2
4980 games collected, 6531 errors
KeyError in id: 3020 for orange player 0
KeyError in id: 3020 for orange player 1
4985 games collected, 6533 errors
KeyError in id: 3022 for orange player 1
KeyError in id: 3022 for orange player 2
KeyError in id: 3022 for blue player 0
KeyError in id: 3022 for blue player 1
KeyError in id: 3022 for blue player 2
KeyError in id: 3023 for orange player 0
KeyError in id: 3025 for orange player 0
KeyError in id: 3025 for orange player 1
KeyError in id: 3025 for orange player 2
KeyError in id: 3025 for blue player 0
KeyError in id: 3025 for blue player 1
KeyError in id: 3025 for blue player 2
4995 games collected, 6545 e

KeyError in id: 3097 for orange player 1
5095 games collected, 6716 errors
KeyError in id: 3098 for orange player 0
3100 id's have been checked, 5098 replays collected
KeyError in id: 3101 for orange player 0
KeyError in id: 3102 for orange player 0
KeyError in id: 3102 for orange player 1
KeyError in id: 3102 for orange player 2
KeyError in id: 3102 for blue player 0
KeyError in id: 3102 for blue player 1
KeyError in id: 3102 for blue player 2
KeyError in id: 3109 for orange player 0
KeyError in id: 3109 for orange player 1
KeyError in id: 3110 for orange player 0
KeyError in id: 3110 for orange player 1
5120 games collected, 6728 errors
KeyError in id: 3111 for orange player 0
KeyError in id: 3111 for orange player 1
KeyError in id: 3111 for orange player 2
KeyError in id: 3111 for blue player 0
KeyError in id: 3111 for blue player 1
KeyError in id: 3111 for blue player 2
5120 games collected, 6734 errors
KeyError in id: 3112 for orange player 0
KeyError in id: 3112 for orange player

In [233]:
replays = replays[0]

In [241]:
replays.player_rank

e8447c7649ce4c6cb88152d742689bd0           Silver II Division 2
TophTophRhodes                             Silver II Division 2
ff88618ff2010900                           Silver II Division 2
76561199080906162                         Platinum I Division 1
GonzoTacoBrown                            Silver III Division 4
                                               ...             
alonso_aco20                               Bronze II Division 4
KnownCart2595                              Bronze II Division 4
ccd73be9c9024cdeb7f266dae0511bf5    Grand Champion I Division 1
b9c630f7bf8444a096ef7f35743deb89    Grand Champion I Division 1
76561198973035879                   Grand Champion I Division 1
Name: player_rank, Length: 5134, dtype: object

In [242]:
replays.to_csv('../data/all_replays.csv')

In [109]:
res = requests.get(f'https://ballchasing.com/api/replays/{replay_id_list[4000]}',
                  headers={
                      'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

data = res.json()
data

{'chat': {'Mountain': 'Nice One!', 'YOU': 'OMG!'},
 'error': 'not found',
 'request_id': '56AB5B86:D77D_A2379E0D:01BB_644E6DFA_B6BC6:820B2',
 'server': 'hzprod-front-2'}